In [4]:
import numpy as np
import pandas as pd
# from textblob import TextBlob
# from wordcloud import WordCloud
import re
import matplotlib.pyplot as plt
import string 
import nltk
import warnings
%matplotlib inline
warnings.filterwarnings("ignore")
plt.style.use('fivethirtyeight')

In [2]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, precision_recall_curve


import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from spacy.lang.en import English
from spacy.lang.en.stop_words import STOP_WORDS
lemma = WordNetLemmatizer()

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Bidirectional, LSTM, Dropout, BatchNormalization
from keras.layers.embeddings import Embedding

In [5]:
df=pd.read_csv("BSNL Twitter Sentiment analysis.csv")
df.head(5)

,ID,Flag,Final_Result,User Tweet,Username,Title,Scores,Created at,Description,Followers Count,Retweet,Location,Source,Image URL,Translated Tweet,Result
0,1,1.0,@NiteshRPradhan @BSNLCorporate @BSNL_KOTD Dear...,@NiteshRPradhan @BSNLCorporate @BSNL_KOTD Dear...,Sikkimbsnl,1440663624318545934,0.500000,09/22/2021,Sikkim BSNL,34.0,0.0,"Gangtok, India",NaN,NaN,NaN,@NiteshRPradhan @BSNLCorporate @BSNL_KOTD Dear...
1,2,1.0,hello %23BSNL,hello %23BSNL,PallaviUike,1440663660439883776,0.500000,09/22/2021,NaN,1.0,0.0,NaN,NaN,NaN,NaN,hello %23BSNL
2,3,1.0,@DoT_India @TRAI @BSNLCorporate\nAs per new ru...,@DoT_India @TRAI @BSNLCorporate\nAs per new ru...,SagarTa04526603,1440667266715058193,0.946512,09/22/2021,मुझे किसी भी धर्म से कोई मतलब नहीं हे ल...,46.0,0.0,NaN,NaN,NaN,NaN,@DoT_India @TRAI @BSNLCorporate\nAs per new ru...
3,4,1.0,RT @drharshvardhan: Modi govt simplified KYC p...,RT @drharshvardhan: मोदी सरकार ने सरल की दूरसं...,RobDehradun,1440664930689056768,0.500000,09/22/2021,official Twitter account प्रादेशिक लोक संपर्क ...,359.0,25.0,"Dehradun, Uttarakhand",NaN,NaN,NaN,RT @drharshvardhan: Modi govt simplified KYC p...
4,5,1.0,RT @AshwiniVaishnaw: KYC process simplified fo...,RT @AshwiniVaishnaw: KYC process simplified fo...,MuskaanRoy4,1440666110924185611,0.995655,09/22/2021,Political and cooking,29.0,274.0,Teliamura Netajinagar,NaN,NaN,NaN,RT @AshwiniVaishnaw: KYC process simplified fo...


In [7]:
#Clean User Tweet
#Create function to clean tweet
def cleanTxt(text):
    text = re.sub(r'(?<=^|(?<=[^a-zA-Z0-9-_\.]))@([A-Za-z]+[A-Za-z0-9-_]+)', '', text) #Removes @ mentions
    text = re.sub(r'#', '', text) #removing hastags
    text = re.sub(r'RT[\s]+', '', text) #removing RT
    text = re.sub(r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))", '',text) #removing Hyperlink
    text = re.sub(r'%23','' , text) 
    text = re.sub(r'\n','' , text) 
    text = re.sub(r':','' , text) 
    return text
df['User Tweet'] = df['User Tweet'].astype(str)

df['Tweet']= df['User Tweet'].apply(cleanTxt)

In [8]:
eng_tweets_df=df[df['Source']=='en']
eng_tweets_df['User_Tweet']=eng_tweets_df['User Tweet'].apply(cleanTxt)
eng_tweets_df['User_Tweet']

7             need your intervention here. I am facing ...
8          We may lose our job due to this frequent fib...
9         Thanks but it's been like more than 3 weeks i...
10         Landline phone 02833-285711 is not working f...
11       In a move to enable contactless customer-centr...
                               ...                        
11907    @1988Singhkomal Now MobileRecharge will have a...
11908     04 Feb. 2022 , Jaipur || Today, Sh. Sandeep G...
11909     Power your STAUPs with BSNL EB services.For i...
11911     04 Feb. 2022 , Jaipur || Today, Sh. Sandeep G...
11912                         Only BSNL can do this..BSNL 
Name: User_Tweet, Length: 7664, dtype: object

In [9]:
eng_tweets_df[eng_tweets_df['Scores'].isna()]

,ID,Flag,Final_Result,User Tweet,Username,Title,Scores,Created at,Description,Followers Count,Retweet,Location,Source,Image URL,Translated Tweet,Result,Tweet,User_Tweet


In [10]:
embeddings_index = dict()
f = open('V:/ML_projects/InterviewTask/NLP_pipeline/pretrain_embeddings/glove.6B.50d.txt',encoding='utf8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

In [11]:
#Creating Embeddings for the titles

max_len_title = eng_tweets_df.User_Tweet.apply(lambda x: len(x.split())).max()

tok_title = Tokenizer()
tok_title.fit_on_texts(eng_tweets_df.User_Tweet)

vocab_size_title = len(tok_title.word_index) + 1

encoded_title = tok_title.texts_to_sequences(eng_tweets_df.User_Tweet)

padded_title = pad_sequences(encoded_title, maxlen=max_len_title, padding='post')

vocab_size_title = len(tok_title.word_index) + 1

title_embedding_matrix = np.zeros((vocab_size_title, 50))
for word, i in tok_title.word_index.items():
    t_embedding_vector = embeddings_index.get(word)
    if t_embedding_vector is not None:
        title_embedding_matrix[i] = t_embedding_vector

In [12]:
import math
from math import exp
import keras
# import tensorflow as tf

from keras import backend as K

def mod_tanh(x):
    return K.tanh(0.6*x)


# config = tf.compat.v1.ConfigProto( device_count = {'GPU': 1 , 'CPU': 56} ) 
# sess = tf.compat.v1.Session(config=config) 
# tf.compat.v1.keras.backend.set_session(sess)

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D

In [13]:

title_model = Sequential()
title_model.add(Embedding(vocab_size_title, 50, input_length=max_len_title, weights=[title_embedding_matrix], trainable=True))
title_model.add(Conv1D(filters=7, kernel_size=8, activation='relu'))
title_model.add(MaxPooling1D(pool_size=2))
title_model.add(Flatten())
print(title_model.summary())
title_model.add(Dense(8, activation='relu'))
title_model.add(Dense(1, activation=mod_tanh))
title_model.compile(loss='mse', optimizer='adam', metrics=['mse', 'mae'])
title_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 55, 50)            486250    
_________________________________________________________________
conv1d (Conv1D)              (None, 48, 7)             2807      
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 24, 7)             0         
_________________________________________________________________
flatten (Flatten)            (None, 168)               0         
Total params: 489,057
Trainable params: 489,057
Non-trainable params: 0
_________________________________________________________________
None
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 55, 50)            486250    
_________________________

In [14]:

X_train_title = eng_tweets_df.loc[:,'Tweet'].values
y_train_title = eng_tweets_df.loc[:,['Scores']].values

x_train_title, x_valid_title, Y_train_title, y_valid_title = train_test_split(padded_title, y_train_title, shuffle = True, test_size = 0.15)

title_model.fit(x_train_title, Y_train_title, epochs = 10,verbose=1)

Epoch 1/10
204/204 [==============================] - 16s 30ms/step - loss: 0.1171 - mse: 0.1171 - mae: 0.2699
Epoch 2/10
204/204 [==============================] - 3s 16ms/step - loss: 0.0432 - mse: 0.0432 - mae: 0.1515
Epoch 3/10
204/204 [==============================] - 4s 19ms/step - loss: 0.0276 - mse: 0.0276 - mae: 0.1120
Epoch 4/10
204/204 [==============================] - 3s 15ms/step - loss: 0.0176 - mse: 0.0176 - mae: 0.0856
Epoch 5/10
204/204 [==============================] - 3s 16ms/step - loss: 0.0109 - mse: 0.0109 - mae: 0.0656
Epoch 6/10
204/204 [==============================] - 3s 14ms/step - loss: 0.0077 - mse: 0.0077 - mae: 0.0527
Epoch 7/10
204/204 [==============================] - 3s 14ms/step - loss: 0.0056 - mse: 0.0056 - mae: 0.0446
Epoch 8/10
204/204 [==============================] - 3s 14ms/step - loss: 0.0046 - mse: 0.0046 - mae: 0.0413
Epoch 9/10
204/204 [==============================] - 3s 16ms/step - loss: 0.0041 - mse: 0.0041 - mae: 0.0399
Epoch 10/